<a href="https://colab.research.google.com/github/sukmoonlee/JupyterNotebook/blob/main/2020/KrCERT_to_RSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## KISA KrCERT 보안공지 to RSS
 - https://www.krcert.or.kr/data/secNoticeList.do 보안공지를 RSS로 제공
 

### 필요 라이브러리 호출

In [1]:
!pip install feedgen

     |████████████████████████████████| 225kB 5.0MB/s 
  Created wheel for feedgen: filename=feedgen-0.9.0-py2.py3-none-any.whl size=44384 sha256=a0819f46b4ad67616b3b4fb3eb933226ec37fc4612b633faaba36ce5eea09eb8
  Stored in directory: /root/.cache/pip/wheels/ae/95/5c/d19cc10ed89cfd3bc24e3e8b5da038084ba1c909e959cd5a9e
Successfully built feedgen


In [2]:
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup

import datetime
from feedgen.feed import FeedGenerator

### 공지사항 정보 추출하기

In [3]:
url = "https://www.krcert.or.kr/data/secNoticeList.do"
data = req.urlopen(url).read()
soup = BeautifulSoup(data, "html.parser")

In [4]:
rss_df = pd.DataFrame()

table_rows = soup.select("table.basicList > tbody > tr")
for table_row in table_rows:
  table_datas = table_row.select("td")

  FLAG_CNT=0
  for table_data in table_datas:
    FLAG_CNT = FLAG_CNT + 1
    try:
      if FLAG_CNT==1 :
        idx = table_data.string.strip()
      if FLAG_CNT==2 :
        url = "https://www.krcert.or.kr" + table_data.select_one('a')['href']
        title = table_data.select_one('a').string.strip()       
      if FLAG_CNT==5 :
        cdate = table_data.string.strip().replace('.','-')
    except:
      continue
  
  item_data = req.urlopen(url).read()
  item_soup = BeautifulSoup(item_data, "html.parser")
  try:
    content = item_soup.select("table.basicView > tbody > tr > td > table > tbody > tr > td")
    article_content = content[0].get_text()
  except:
    article_content=""

  d=pd.DataFrame()
  d.loc[idx, 'title'] = title
  d.loc[idx, 'link'] = url
  d.loc[idx, 'cdate'] = cdate
  d.loc[idx, 'descipt'] = article_content
  rss_df = rss_df.append(d)

rss_df['cdate'] = pd.to_datetime(rss_df['cdate'])
rss_df['cdate'] = rss_df['cdate'].dt.tz_localize('Asia/Seoul')
rss_df

,title,link,cdate,descipt
1406,Xerox DocuShare 보안 업데이트 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-12-04 00:00:00+09:00,□ 개요\r\n o Xerox 社는 DocuShare 제품에서 발생하는 취약점을 해...
1405,VMware 제품군 보안 업데이트 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-12-04 00:00:00+09:00,□ 개요\r\n o VMware 社는 자사 제품에서 발생하는 취약점을 해결한 보안 ...
1404,Mozilla Thunderbird 보안 업데이트 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-12-04 00:00:00+09:00,□ 개요\r\n o 모질라 재단은 Thunderbird의 취약점을 해결한 보안 업데...
1403,Adobe Flash Player 기술지원 종료 관련 보안 주의 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-12-02 00:00:00+09:00,"□ 개요\r\n o 2020년 12월 31일, Adobe 社의 Flash Playe..."
1402,HP Device Manager 취약점 보안 업데이트 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-12-01 00:00:00+09:00,□ 개요\r\n o HP 社는 자사 제품에서 발생하는 취약점을 해결한 보안 업데이트...
1401,VMware 제품군 보안 주의 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-11-26 00:00:00+09:00,□ 개요\r\n o VMware 社는 자사 제품에서 발생하는 취약점 주의 발표 [1...
1400,Cisco 제품 취약점 보안 업데이트 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-11-23 00:00:00+09:00,□ 개요\r\n o Cisco社는 자사 제품의 취약점을 해결한 보안 업데이트 공지 ...
1399,랜섬웨어 감염 확산에 따른 기업 보안점검 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-11-22 00:00:00+09:00,□ 개요\r\n o 최근 기업 대상 랜섬웨어 감염 및 정보유출 사고가 지속적으로 발...
1398,위즈베라 통합 보안 설치 프로그램 베라포트(veraport) 보안 업데이트 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-11-18 00:00:00+09:00,□ 개요\r\n o 위즈베라(Wizvera)社의 통합 보안 설치 프로그램 베라포트(...
1397,Apple 제품군 보안 업데이트 권고,https://www.krcert.or.kr/data/secNoticeView.do...,2020-11-16 00:00:00+09:00,□ 개요\r\n o Apple社는 자사 제품에서 발생하는 취약점을 해결한 보안 업데...


### RSS Feed 만들기

In [5]:
fg = FeedGenerator()
fg.id('https://www.krcert.or.kr/data/secNoticeList.do')
fg.title('KrCERT 보안공지')
fg.author( {'name':'Sukmoon Lee','email':'smlee@sukmoonlee.com'} )
fg.link( href='https://www.krcert.or.kr/data/secNoticeList.do', rel='alternate' )
fg.subtitle('KrCERT 보안공지 RSS')
fg.language('kr')

for idx in rss_df.index:
  fe = fg.add_entry()
  fe.id(idx)
  fe.title(rss_df.loc[idx, 'title'])
  fe.link(href=rss_df.loc[idx, 'link'])
  fe.description(rss_df.loc[idx, 'descipt'])
  fe.pubDate(pubDate=rss_df.loc[idx, 'cdate'])

In [10]:
fg.rss_file('krcert_rss.xml', pretty=True) # Write the RSS feed to a file
!cat krcert_rss.xml |head

<?xml version='1.0' encoding='UTF-8'?>
<rss xmlns:atom="http://www.w3.org/2005/Atom" xmlns:content="http://purl.org/rss/1.0/modules/content/" version="2.0">
  <channel>
    <title>KrCERT 보안공지</title>
    <link>https://www.krcert.or.kr/data/secNoticeList.do</link>
    <description>KrCERT 보안공지 RSS</description>
    <docs>http://www.rssboard.org/rss-specification</docs>
    <generator>python-feedgen</generator>
    <language>kr</language>
    <lastBuildDate>Mon, 07 Dec 2020 07:32:35 +0000</lastBuildDate>
